In [1]:
import sys
from traceback import format_exc
sys.path.append("../")

In [3]:
from code_tracer import CodeTracer
from report_builder import ReportBuilder

In [6]:
def parsecode(code):
    try:
        if not code or len(code.strip()) == 0:
            return {"error": "code is null or empty"}

        tracer = CodeTracer()
        tracer.max_width = 200000
        code_report = tracer.trace_code(code, dump=False)
        return code_report
    
    except Exception as e:
        exc = format_exc()
        print(exc)
        return None

In [8]:
code = """

i = 0
i += 1

s = "a"
s += "b"
s += "c"

l = []
l.append(i)
l.append(s)
l.append(l)
"""

parsecode(code)

Traceback (most recent call last):
  File "<ipython-input-6-338610e8f599>", line 3, in parsecode
    code = request.form["code"]
NameError: name 'request' is not defined

